In [ ]:
import torch
from torch import nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torch.amp import autocast, GradScaler
import torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau

from transformers import VivitConfig, VivitModel

from tqdm.notebook import tqdm
# from tqdm import tqdm
from PIL import Image
import json
import os
import numpy as np
import random

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from data.rwth_phoenix import rwth_phoenix
from data.dataloader import collate_fn

from models.Vivit_r3d_tf import ViViT_SLR

from utils.visualization import show_sequence
from utils.mem_optim import preload_data

from train_test.train_test_step import decode_token, train_step, test_step

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(42)

In [ ]:
train_csv = "/kaggle/input/hmmmmmm/phoenix/annotations/manual/train.corpus.csv"
test_csv = "/kaggle/input/hmmmmmm/phoenix/annotations/manual/test.corpus.csv"
dev_csv = "/kaggle/input/hmmmmmm/phoenix/annotations/manual/dev.corpus.csv"

train_paths = "/kaggle/input/hmmmmmm/phoenix/fullFrame-210x260px/train"
test_paths = "/kaggle/input/hmmmmmm/phoenix/fullFrame-210x260px/test"
dev_paths =  "/kaggle/input/hmmmmmm/phoenix/fullFrame-210x260px/dev"

In [10]:
max_frames = 32

In [ ]:
num_workers = 4
batch_size = 2

In [5]:
word_to_idx = { '<p>':0, '<s>':1, '</s>':2}
idx_to_word = ['<p>', '<s>', '</s>']

arr_train = np.loadtxt(train_csv, delimiter='|', dtype='str')
arr_train = np.delete(arr_train,0,0)
arr_test = np.loadtxt(test_csv, delimiter='|', dtype='str')
arr_test = np.delete(arr_test,0,0)
arr_dev = np.loadtxt(dev_csv, delimiter='|', dtype='str')
arr_dev = np.delete(arr_dev,0,0)

arr = np.concatenate((arr_train, arr_test, arr_dev), axis=0)

for sentence in arr:
    for word in sentence[3].split(' '):
        if word not in idx_to_word:
            idx_to_word.append(word)
            word_to_idx[word] = len(idx_to_word)-1

In [6]:
len(idx_to_word), len(word_to_idx)

(1299, 1299)

In [7]:
image_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomCrop((248,200)),
    transforms.Resize((224,224)),
    transforms.RandomRotation((-5,5))
])

video_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(0.5)
])

In [8]:
image_test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
])

video_test_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(0)
])

In [ ]:
train_dataset = rwth_phoenix(csv=train_csv,
                       data_path=train_paths,
                        frame_transform=image_transform , video_transform=video_transform, input_fps=25, output_fps=list(range(4,10)), max_frames=max_frames, stride=0, word_dict=word_to_idx)

test_dataset = rwth_phoenix(csv=test_csv,
                       data_path=train_paths,
                        frame_transform=image_test_transform , video_transform=video_test_transform, input_fps=25, output_fps=list(range(4,10)), max_frames=max_frames, stride=0, word_dict=word_to_idx)

dev_dataset = rwth_phoenix(csv=dev_csv,
                       data_path=dev_paths,
                        frame_transform=image_test_transform , video_transform=video_test_transform, input_fps=25, output_fps=list(range(4,10)), max_frames=max_frames, stride=0, word_dict=word_to_idx)

In [14]:
train_dataset[0][0].shape

torch.Size([32, 3, 224, 224])

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size, collate_fn=collate_fn,prefetch_factor=5,num_workers=num_workers, pin_memory=True)
test_dataloader = DataLoader(dataset=test_dataset, shuffle=False, batch_size=batch_size, collate_fn=collate_fn,prefetch_factor=5,num_workers=num_workers, pin_memory=True)
dev_dataloader = DataLoader(dataset=dev_dataset, shuffle=False, batch_size=batch_size, collate_fn=collate_fn,prefetch_factor=5,num_workers=num_workers, pin_memory=True)

In [ ]:
train_data = preload_data(train_dataloader, 'train')
test_data = preload_data(test_dataloader, 'test')
dev_data = preload_data(dev_dataloader, 'dev')

In [ ]:
lr = 1e-4
dropout = 0.1

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = "cuda"

model = ViViT_SLR(
                 vocab_size=len(idx_to_word),
                 d_model=768,
                 nhead=8,
                 num_decoder_layers=4,
                 dim_feedforward=3072,
                 dropout=dropout,
                 batch_first=True,
                 num_heads=4,
                 pad_token=0,
                 sos_token=1,
                 eos_token=2,
                 residual_ratio=0,
                 max_pred=512).to(device)
# checkpoint = torch.load(f="/kaggle/input/vvocr_final/pytorch/default/1/VVOCR.pth")
# new_state_dict = checkpoint["model_state_dict"]
# model.load_state_dict(new_state_dict, strict=True)
# model = nn.DataParallel(model)
# checkpoint = torch.load(f="/kaggle/input/vvocr/pytorch/default/1/VVOCR.pth")
# from collections import OrderedDict

# new_state_dict = OrderedDict()
# for k, v in checkpoint['model_state_dict'].items():
#     new_key = k.replace("module.", "")  # strip 'module.' prefix
#     new_state_dict[new_key] = v
# model.load_state_dict(new_state_dict, strict=True)
loss_fn = nn.CrossEntropyLoss(ignore_index=0, label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=2,
    min_lr=1e-6,
)

In [ ]:
epochs = 10
torch.manual_seed(42)
torch.cuda.manual_seed(42)
for epoch in tqdm(range(epochs), 'epochs'):
  train_step(model=model,
             optimizer=optimizer,
             loss_fn=loss_fn,
             epoch=epoch,
             dataloader=train_data)
  test_step(model=model,
            loss_fn=loss_fn,
            epoch=epoch,
            dataloader=test_data,
            scheduler=scheduler
           )